## Lập trình xử lý dữ liệu - Nhóm 7: Đánh đâu lỗ đó
### Notebook Áp dụng học máy để phân loại cảm xúc bài viết
---

### Part 1: Preparation

Ta thực hiện import các thư viện cần thiết gồm pandas, string, sklearn và demoji.

pandas: Thư viện này được sử dụng để xử lý và phân tích dữ liệu, đặc biệt là khi làm việc với các cấu trúc dữ liệu như DataFrame, giúp dễ dàng quản lý và thao tác dữ liệu dạng bảng.

string: Thư viện này cung cấp các chức năng hữu ích để thao tác với chuỗi ký tự trong Python, ví dụ như loại bỏ các dấu câu hoặc kiểm tra các ký tự cụ thể trong chuỗi.

sklearn: Thư viện này hỗ trợ các công cụ máy học, bao gồm các mô hình phân loại như RandomForestClassifier. CountVectorizer từ sklearn được sử dụng để chuyển đổi văn bản thành các đặc trưng có thể sử dụng cho mô hình học máy. train_test_split giúp chia tập dữ liệu thành tập huấn luyện và tập kiểm tra.

demoji: Thư viện này giúp xử lý và loại bỏ các emoji trong dữ liệu văn bản, giúp làm sạch dữ liệu trước khi phân tích.

Trong phần phân tích này, ta sử dụng CountVectorizer để chuyển đổi nội dung bài viết thành các vector đặc trưng (sử dụng phương pháp "bag of words"). Sau đó, mô hình học máy RandomForestClassifier được áp dụng để phân loại các bài viết.

In [86]:
import pandas as pd

import string 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import demoji 



In [87]:
demoji.download_codes()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21616\2299328559.py:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [88]:
posts_df = pd.read_csv('cleaned_posts.csv')
posts_df.head()

,postID,originalContent,date,link,sentiment,totalLikes,totalReplies,replyToPostID,taggedSymbols,username,userid,totalImages,totalSymbols
0,29104030,Vkl luôn,2024-11-06T21:39:23.267+07:00,NaN,0,1,0,NaN,"[{'symb': '^DJI', 'price': 43496.43}]",Hoàng,5ededf24-12f7-41d9-b390-08ff631fc275,0,1
1,29104026,Đ.ịt cụ thằng Khải Trần hô VNI sập về 900 lần ...,2024-11-06T21:39:05.22+07:00,NaN,0,0,0,NaN,[],Datbg10,73778567-ec61-43eb-b3a0-b4a651b8bd3f,0,0
2,29104024,Tăng 1 phát bằng vn làm hai mấy năm :)),2024-11-06T21:39:04.077+07:00,NaN,0,12,2,NaN,"[{'symb': '^DJI', 'price': 43506.86}]",Trung Tuyến,5d597f38-3b24-4f40-952c-2b3f9be8e7d5,0,1
3,29104021,Má đáng full tiền...đau,2024-11-06T21:38:52.277+07:00,NaN,0,0,1,NaN,"[{'symb': 'VNINDEX', 'price': 1261.28}]",Duc Nguyen,225659c6-cf18-4e93-aa39-a294bae5b784,1,1
4,29104019,Gap khủng long,2024-11-06T21:38:49.99+07:00,NaN,0,2,0,NaN,"[{'symb': '^DJI', 'price': 43499.54}]",BINH NHI,da8ebfa2-9cd5-4dbf-84cd-9567f694f681,0,1


Sau khi thực hiện đẩy dữ liệu từ file clean_posts.csv vào dataframe posts_df, ta chỉnh sưa cột sentiment từ 1, 0 và -1 lần lượt thành 'positive', 'neutral' và 'negative' tương ứng giúp dễ đọc và phân tích hơn.

In [89]:
posts_df['sentiment'] = posts_df['sentiment'].apply(lambda x: 'positive' if x == 1
                                                        else 'negative' if x == -1 else 'neutral')

Ta tải danh sách các từ dừng (stop words) từ một file CSV và chuyển chúng thành một tập hợp (set) để tối ưu hóa việc tra cứu. Hàm preprocess_text_optimized xử lý văn bản đầu vào bằng cách chuyển đổi thành chữ thường, loại bỏ dấu câu và các từ dừng, nhằm làm sạch dữ liệu trước khi phân tích. Kết quả là văn bản đã được chuẩn hóa, giúp các bước xử lý tiếp theo trở nên hiệu quả hơn.

Stop words là các từ phổ biến, không mang nhiều ý nghĩa trong việc phân tích ngữ nghĩa của văn bản, như "và", "hoặc", "của", "the", "is", "are",... Những từ này thường không đóng góp nhiều trong việc phân loại hay tìm kiếm thông tin, vì vậy chúng được loại bỏ trong quá trình tiền xử lý dữ liệu. Trong đoạn code, các từ dừng được tải từ file CSV chứa danh sách các từ dừng tiếng Việt và sẽ được loại bỏ khỏi văn bản trong bước tiền xử lý.

In [90]:
stop_words = pd.read_csv('vietnamese-stopwords.csv')
stop_words = stop_words['word'].tolist()
stop_words = set(stop_words)

def preprocess_text_optimized(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return ' '.join(word for word in text.split() if word not in stop_words)

In [91]:
post_content = posts_df[['originalContent', 'sentiment']]

post_content['originalContent'] = post_content['originalContent'].apply(preprocess_text_optimized)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21616\1553288514.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_content['originalContent'] = post_content['originalContent'].apply(preprocess_text_optimized)


Ta cũng loại bỏ thêm các bài viết mà chỉ chứa link hoặc các bài viết spam. Đây là những nội dung rác không có tính đóng góp trong việc phân loại cảm xúc. 

Đồng thời ta cũng loại bỏ các emoji ra khỏi bài viết.

In [92]:
# remove contents contains link only
post_content = post_content[~post_content['originalContent'].str.match(r'^\s*(http|https|www\.).*$', na=False)]
post_content.dropna(inplace=True)

# remove all the emojis
post_content['originalContent'] = post_content['originalContent'].apply(demoji.replace)


In [93]:
post_content.head(1)

,originalContent,sentiment
0,vkl,neutral


Đoạn code trên tải danh sách các từ tục tĩu từ file Vietnamese_cursed_words.txt và chuyển chúng thành một tập hợp (set) để dễ dàng tra cứu. Hàm contains_cursed_words nhận vào một văn bản, chuyển văn bản thành chữ thường và tách thành các từ. Sau đó, nó đếm số lượng từ tục tĩu xuất hiện trong văn bản. Nếu có từ tục tĩu, hàm trả về giá trị 1 cùng với số lượng từ tục tĩu; nếu không, hàm trả về 0 và số lượng là 0. Quá trình này giúp phát hiện và đếm các từ tục tĩu trong văn bản, cũng cố cho độ chính xác của model học máy.

In [96]:
cursed_words = pd.read_csv('Vietnamese_cursed_words.txt')
cursed_words = cursed_words['word'].tolist()
cursed_words = set(cursed_words)

def contains_cursed_words(text):
    words = text.lower().split()
    # return number of cursed words in the text
    number_of_cursed_words = sum(1 for word in words if word in cursed_words)
    if number_of_cursed_words > 0:
        return 1, number_of_cursed_words
    return 0, 0

# test function
contains_cursed_words('đệch đéo')
    

(1, 2)

In [97]:
post_content[['contains_cursed_words', 'numberOfCursedWords']] = post_content['originalContent'].apply(
    contains_cursed_words
).apply(pd.Series)

post_content

,originalContent,sentiment,contains_cursed_words,numberOfCursedWords
0,vkl,neutral,1,1
1,địt cụ thằng khải trần hô vni sập 900 100,neutral,1,1
2,1 phát vn hai mấy,neutral,0,0
3,má full tiềnđau,neutral,0,0
4,gap khủng long,neutral,0,0
...,...,...,...,...
272974,vc,neutral,0,0
272975,chết dập chết dụi hic,neutral,1,2
272976,múc rũ đỉnh,neutral,0,0
272977,đừng mong giá rẻ mấy ní đỏ ko mua xanh line,neutral,0,0


In [98]:
# check if function works correctly
post_content[post_content['numberOfCursedWords'] > 1].head(5)

,originalContent,sentiment,contains_cursed_words,numberOfCursedWords
80,vãi lồn,neutral,1,2
116,ae đc xác thằng long quan official trôi dạt kh...,neutral,1,2
193,đọc mãi báo đéo công văn quyết định thu hồi đé...,negative,1,4
344,vre xác 100 thoát vin xác 1 7 vol 35tr tham gi...,neutral,1,3
358,trump đắc cử tổng thống 47 mỹ chứng khoán việt...,neutral,1,2


### Part 2: Training and testing model accuracy.

Cách 1: thực hiện việc xây dựng và đánh giá mô hình phân loại cảm xúc cho các bài viết. 

Đầu tiên, dữ liệu được chia thành hai phần: dữ liệu huấn luyện và dữ liệu kiểm tra, với các bài viết có cảm xúc không phải là "neutral" được dùng để huấn luyện mô hình. 

Văn bản của các bài viết được chuyển đổi thành đặc trưng số bằng cách sử dụng phương pháp TfidfVectorizer để tạo ma trận đặc trưng từ nội dung bài viết. 

Sau đó, các đặc trưng bổ sung như số lượng từ tục tĩu cũng được kết hợp vào ma trận đặc trưng này. Mô hình phân loại được huấn luyện với thuật toán RandomForestClassifier và được đánh giá bằng điểm số chính xác và báo cáo phân loại (classification report), cung cấp các thông số như độ chính xác, độ nhạy, và điểm F1 của mô hình.

In [99]:
import scipy.sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report


train_content = post_content[post_content['sentiment'] != 'neutral']

vectorizer = TfidfVectorizer()
X_text = vectorizer.fit_transform(train_content['originalContent'])
Y = train_content['sentiment']

X_train, X_test, Y_train, Y_test = train_test_split(X_text, Y, test_size = 0.2, stratify = Y, random_state = 42)

X = scipy.sparse.hstack([X_text, scipy.sparse.csr_matrix(train_content[['contains_cursed_words', 'numberOfCursedWords']].values)])

clf = RandomForestClassifier(n_jobs = -1)
clf.fit(X_train, Y_train)

print(clf.score(X_test, Y_test))

y_pred = clf.predict(X_test)
print(classification_report(Y_test, y_pred))

0.7582417582417582
              precision    recall  f1-score   support

    negative       0.71      0.50      0.59      2654
    positive       0.77      0.89      0.83      5081

    accuracy                           0.76      7735
   macro avg       0.74      0.70      0.71      7735
weighted avg       0.75      0.76      0.75      7735



#### Nhận xét:
#### Kết quả phân loại cảm xúc

Mô hình phân tích cảm xúc của bài viết đạt kết quả như sau:

##### 1. **Độ chính xác (Accuracy)**

- **Giải thích**: Độ chính xác là tỷ lệ bài viết được phân loại đúng trong tổng số bài viết.
- **Công thức**: 
  $$
  \text{Accuracy} = \frac{\text{Số lượng phân loại đúng}}{\text{Tổng số bài viết}}
  $$
- **Kết quả**: 0.76 (76%)
  
Mô hình có độ chính xác 76%, cho thấy mô hình có khả năng phân loại đúng tổng thể các bài viết trong tập kiểm tra.

##### 2. **Độ chính xác (Precision)**

- **Giải thích**: Độ chính xác đo lường tỷ lệ các bài viết thực sự thuộc lớp đó trong tất cả các bài viết được phân loại vào lớp đó.
- **Công thức**: 
  $$
  \text{Precision} = \frac{\text{True Positive}}{\text{True Positive} + \text{False Positive}}
  $$
- **Kết quả**:
  - **Lớp "negative"**: 0.71 (71%)
  - **Lớp "positive"**: 0.77 (77%)
  
Mô hình có độ chính xác khá cao cho lớp "positive" nhưng thấp hơn đối với lớp "negative", cho thấy mô hình ít phân loại sai với lớp "positive".

##### 3. **Độ nhạy (Recall)**

- **Giải thích**: Độ nhạy đo lường khả năng của mô hình trong việc phát hiện tất cả các bài viết thuộc lớp đó.
- **Công thức**: 
  $$
  \text{Recall} = \frac{\text{True Positive}}{\text{True Positive} + \text{False Negative}}
  $$
- **Kết quả**:
  - **Lớp "negative"**: 0.50 (50%)
  - **Lớp "positive"**: 0.89 (89%)

Mô hình có độ nhạy tốt cho lớp "positive", nhưng đối với lớp "negative", độ nhạy thấp cho thấy mô hình chưa phát hiện được hết các bài viết tiêu cực.

##### 4. **Điểm F1 (F1-score)**

- **Giải thích**: Điểm F1 là chỉ số kết hợp giữa độ chính xác và độ nhạy, dùng để đánh giá hiệu suất tổng thể của mô hình.
- **Công thức**: 
  $$
  \text{F1-score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
  $$
- **Kết quả**:
  - **Lớp "negative"**: 0.59
  - **Lớp "positive"**: 0.83
  
Điểm F1 của lớp "positive" cao hơn lớp "negative", cho thấy mô hình phân loại bài viết tích cực tốt hơn tiêu cực.

##### 5. **Chỉ số tổng hợp**

- **Macro avg**: Trung bình các chỉ số cho tất cả các lớp.
  - **Precision**: 0.74
  - **Recall**: 0.70
  - **F1-score**: 0.71

- **Weighted avg**: Trung bình có trọng số của các chỉ số, cân nhắc đến sự phân bổ lớp không đều.
  - **F1-score**: 0.75

---

##### Nhận xét

- Mô hình thể hiện hiệu quả tốt trong việc phân loại các bài viết có cảm xúc tích cực (positive) với độ chính xác và độ nhạy cao.
- Tuy nhiên, mô hình gặp khó khăn trong việc nhận diện các bài viết tiêu cực (negative), với độ nhạy chỉ đạt 50%, cho thấy mô hình bỏ sót một phần lớn các bài viết tiêu cực.
- Điểm F1 trung bình cho thấy mô hình có sự cân bằng giữa độ chính xác và độ nhạy, nhưng vẫn cần cải thiện khả năng nhận diện lớp tiêu cực để đạt được hiệu suất tốt hơn.

---


In [100]:
post_to_classify = posts_df.loc[posts_df['sentiment'] == 'neutral', 'originalContent']
print(f"Number of posts need to classify: {len(post_to_classify)}")

X_post = vectorizer.transform(post_to_classify)
sentiment_predicted = clf.predict(X_post)

new_post_df = posts_df.copy()

new_post_df.loc[new_post_df['sentiment'] == 'neutral', 'sentiment'] = sentiment_predicted
new_post_df.head(1)


Number of posts need to classify: 233586


,postID,originalContent,date,link,sentiment,totalLikes,totalReplies,replyToPostID,taggedSymbols,username,userid,totalImages,totalSymbols
0,29104030,Vkl luôn,2024-11-06T21:39:23.267+07:00,NaN,negative,1,0,NaN,"[{'symb': '^DJI', 'price': 43496.43}]",Hoàng,5ededf24-12f7-41d9-b390-08ff631fc275,0,1


Ta áp dụng mô hình vừa huấn luyện để phân loại các bài viết trung tính(neutral) thành tích cực(positive) và tiêu cực(negative).

In [ ]:
print('Number of negative posts (before)', len(posts_df[posts_df['sentiment'] == 'negative']))
print('Number of positive posts(before)', len(posts_df[posts_df['sentiment'] == 'positive']))
print('Number of negative posts (after)', len(new_post_df[new_post_df['sentiment'] == 'negative']))
print('Number of positive posts(after)', len(new_post_df[new_post_df['sentiment'] == 'positive']))

Number of negative posts (before) 13317
Number of positive posts(before) 26076
Number of negative posts (after) 59765
Number of positive posts(after) 213214


Tiêu đạt độ chính xác tổng thể gần 76%, số lượng bài viết tích cực và tiêu cực sau khi được phân loại lại cho thấy sự chênh lệch quá lớn, số lượng bài viết tích cực trước chỉ gấp đôi nay đã gấp gần 4 lần số bài viết tiêu cực. Điều này cho thấy rõ hạn chế về cách tiếp cận và độ chính xác của mô hình.

---
#### Ta triển khai một cách tiếp cận mới nhằm cải thiện hiệu suất phân loại cảm xúc bằng cách cân bằng dữ liệu, sử dụng cả đặc trưng văn bản và các đặc trưng bổ sung (số lượng từ xấu và sự xuất hiện từ xấu). Các bước chính của mô hình này bao gồm:

1. **Chuẩn bị dữ liệu**:
   - Dữ liệu được chia thành hai phần: `train_content` (dùng để huấn luyện) và `test_content` (dùng để kiểm tra).
   - Để xử lý vấn đề mất cân bằng nhãn giữa các bài viết có cảm xúc "negative" và "positive", các hàng có nhãn "negative" trong tập huấn luyện được nhân bản, giúp cân bằng số lượng mẫu giữa các lớp.

2. **Vector hóa nội dung văn bản**:
   - Văn bản được chuyển đổi thành biểu diễn số thông qua `TfidfVectorizer`, tạo ra các đặc trưng dạng sparse từ nội dung bài viết. 
   - Bộ vector hóa được áp dụng riêng cho tập huấn luyện và tập kiểm tra để đảm bảo tính độc lập của dữ liệu kiểm tra.

3. **Kết hợp đặc trưng bổ sung**:
   - Các đặc trưng bổ sung gồm số lượng từ không phù hợp (`numberOfCursedWords`) và cờ đánh dấu sự xuất hiện của từ không phù hợp (`contains_cursed_words`) được chuyển đổi thành ma trận thưa (`sparse matrix`).
   - Văn bản vector hóa và các đặc trưng bổ sung được kết hợp để tạo ra ma trận đặc trưng đầu vào.

4. **Chia tập huấn luyện và kiểm tra**:
   - Tập huấn luyện tiếp tục được chia thành hai phần: tập huấn luyện thực tế (`X_train`) và tập xác thực (`X_val`) theo tỷ lệ 80-20. Việc này giúp đánh giá hiệu suất mô hình trên dữ liệu chưa thấy trước khi kiểm tra trên tập kiểm tra thực tế.

5. **Huấn luyện mô hình**:
   - Mô hình `RandomForestClassifier` được sử dụng để huấn luyện trên tập dữ liệu huấn luyện đã chuẩn bị.

6. **Đánh giá mô hình**:
   - Hiệu suất mô hình được đánh giá trên cả tập xác thực (`X_val`) và tập kiểm tra (`X_test`) bằng cách sử dụng các chỉ số như `precision`, `recall`, `f1-score` và `accuracy`. Báo cáo phân loại (`classification_report`) cung cấp thông tin chi tiết về hiệu quả phân loại từng lớp.

---

#### Nhận xét về cách tiếp cận mới

- **Cân bằng dữ liệu**: Việc nhân bản các bài viết có nhãn "negative" là một cải tiến quan trọng, giúp giải quyết vấn đề mất cân bằng dữ liệu và đảm bảo mô hình không bị thiên vị đối với lớp "positive".
- **Tích hợp đặc trưng bổ sung**: Kết hợp các đặc trưng bổ sung từ nội dung bài viết (như số lượng từ không phù hợp) với đặc trưng văn bản từ `TfidfVectorizer` giúp mô hình tận dụng tốt hơn thông tin phi ngôn ngữ để cải thiện hiệu quả phân loại.
- **Đánh giá hiệu suất**: Việc đánh giá trên cả tập xác thực và tập kiểm tra giúp kiểm chứng khả năng tổng quát hóa của mô hình, đảm bảo rằng nó hoạt động tốt trên dữ liệu chưa thấy trước.

---

#### So sánh với cách tiếp cận cũ

So với cách tiếp cận trước đó:
- **Cải thiện cân bằng mẫu**: Việc nhân bản dữ liệu tiêu cực là một điểm mới, cải thiện khả năng học của mô hình với lớp thiểu số.
- **Tách biệt tập kiểm tra**: Phương pháp này tách riêng tập kiểm tra ngay từ đầu, tránh việc dữ liệu kiểm tra bị ảnh hưởng bởi quá trình huấn luyện và xác thực, đảm bảo đánh giá mô hình khách quan hơn.

Với cách tiếp cận này, mô hình có tiềm năng đạt được kết quả tốt hơn nhờ sử dụng đầy đủ thông tin trong dữ liệu và giải quyết vấn đề mất cân bằng lớp.

In [102]:
import scipy.sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd

# Separate train and test content
train_content = post_content[post_content['sentiment'] != 'neutral']
test_content = post_content[post_content['sentiment'] != 'neutral']

# Duplicate rows that sentiment is negative to balance sample size
train_content = pd.concat([train_content, train_content[train_content['sentiment'] == 'negative']])

# Vectorize the text content
vectorizer = TfidfVectorizer()
X_text_train = vectorizer.fit_transform(train_content['originalContent'])
X_text_test = vectorizer.transform(test_content['originalContent'])

# Additional features
X_train_additional = scipy.sparse.csr_matrix(train_content[['contains_cursed_words', 'numberOfCursedWords']].values)
X_test_additional = scipy.sparse.csr_matrix(test_content[['contains_cursed_words', 'numberOfCursedWords']].values)

# Combine text features and additional features
X_train = scipy.sparse.hstack([X_text_train, X_train_additional])
X_test = scipy.sparse.hstack([X_text_test, X_test_additional])

# Target labels
Y_train = train_content['sentiment']
Y_test = test_content['sentiment']

# Train-Test Split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, stratify=Y_train, random_state=42)

# Model training
clf = RandomForestClassifier(n_jobs=-1, random_state=42)
clf.fit(X_train, Y_train)

# Evaluate on validation set
y_pred = clf.predict(X_val)
print("Validation Classification Report:")
print(classification_report(Y_val, y_pred))

# Evaluate on test set
y_pred_test = clf.predict(X_test)
print("Test Classification Report:")
print(classification_report(Y_test, y_pred_test))



Validation Classification Report:
              precision    recall  f1-score   support

    negative       0.82      0.93      0.87      5308
    positive       0.91      0.79      0.85      5081

    accuracy                           0.86     10389
   macro avg       0.87      0.86      0.86     10389
weighted avg       0.87      0.86      0.86     10389

Test Classification Report:
              precision    recall  f1-score   support

    negative       0.89      0.98      0.93     13268
    positive       0.99      0.94      0.96     25405

    accuracy                           0.95     38673
   macro avg       0.94      0.96      0.95     38673
weighted avg       0.96      0.95      0.95     38673



#### Nhận xét kết quả mô hình

##### 1. **Báo cáo trên tập xác thực (Validation Classification Report)**:
   - **Độ chính xác (Accuracy):** Mô hình đạt **86%**, cho thấy hiệu suất khá tốt trong việc dự đoán cảm xúc trên tập dữ liệu xác thực.
   - **Precision và Recall:**
     - **Negative:** 
       - Precision: 82%, nghĩa là trong tất cả các dự đoán "negative", 82% là chính xác.
       - Recall: 93%, cho thấy mô hình đã phát hiện hầu hết các bài viết thuộc lớp "negative".
     - **Positive:** 
       - Precision: 91%, mô hình rất chính xác trong việc dự đoán "positive".
       - Recall: 79%, thấp hơn "negative", cho thấy mô hình bỏ sót một số bài viết thuộc lớp "positive".
   - **F1-Score:** Trung bình hài hòa của Precision và Recall, lần lượt là **87%** cho "negative" và **85%** cho "positive". Kết quả này phản ánh sự cân đối tương đối tốt giữa hai chỉ số trên.
   - **Nhận xét:** Hiệu suất trên tập xác thực cho thấy mô hình học tốt hơn với lớp "negative" so với "positive", thể hiện qua Recall cao hơn cho "negative". Tuy nhiên, sự chênh lệch này có thể dẫn đến một chút thiên vị.

##### 2. **Báo cáo trên tập kiểm tra (Test Classification Report)**:
   - **Độ chính xác (Accuracy):** Mô hình đạt **95%**, cải thiện đáng kể so với tập xác thực, chứng tỏ mô hình tổng quát hóa tốt khi áp dụng trên dữ liệu chưa thấy trước.
   - **Precision và Recall:**
     - **Negative:**
       - Precision: 89%, nghĩa là trong các dự đoán "negative", 89% là đúng.
       - Recall: 98%, rất cao, chứng tỏ mô hình nhận diện hầu hết các bài viết "negative".
     - **Positive:**
       - Precision: 99%, cho thấy gần như tất cả các dự đoán "positive" đều chính xác.
       - Recall: 94%, hơi thấp hơn một chút, nhưng vẫn rất tốt.
   - **F1-Score:** Lớp "positive" đạt **96%**, cao hơn lớp "negative" (**93%**), phản ánh sự cân đối tốt hơn trên tập kiểm tra.
   - **Nhận xét:** Kết quả trên tập kiểm tra rất ấn tượng, với độ chính xác và các chỉ số khác đều cao. Điều này có thể được giải thích bởi việc mô hình đã được huấn luyện trên một tập dữ liệu được cân bằng tốt hơn và tích hợp các đặc trưng bổ sung.

---

##### **So sánh giữa tập xác thực và tập kiểm tra**:
   - **Độ chính xác:** Tăng từ **86%** trên tập xác thực lên **95%** trên tập kiểm tra, cho thấy sự cải thiện rõ rệt trong dự đoán.
   - **Hiệu suất từng lớp:** 
     - Lớp "negative" cải thiện về Precision và Recall trên tập kiểm tra.
     - Lớp "positive" cũng cho thấy Precision và F1-score rất cao, nhưng Recall giảm nhẹ so với tập xác thực.
   - **Nhận xét tổng thể:** Mặc dù có sự khác biệt giữa các chỉ số trên tập xác thực và kiểm tra, mô hình tổng quát hóa tốt và có khả năng phân loại chính xác cảm xúc của các bài viết, đặc biệt trên tập kiểm tra lớn hơn.

---

##### Kết luận
Mô hình hoạt động hiệu quả trên cả hai tập dữ liệu. Kết quả trên tập kiểm tra đặc biệt tốt, đạt độ chính xác cao và cân đối tốt giữa hai lớp cảm xúc. Với Recall cao cho lớp "negative" và Precision vượt trội cho lớp "positive", mô hình có tiềm năng áp dụng thực tế trong việc phân loại cảm xúc. Tuy nhiên, cần lưu ý rằng mô hình có thể bỏ sót một số bài viết thuộc lớp "positive" trên tập xác thực, và điều này cần được cải thiện thêm thông qua các phương pháp như tinh chỉnh siêu tham số hoặc bổ sung dữ liệu.

In [103]:
# Apply the model to classify the sentiment of neutral posts
post_to_classify = post_content.loc[post_content['sentiment'] == 'neutral', ['originalContent', 'contains_cursed_words', 'numberOfCursedWords']]
print(f"Number of posts need to classify: {len(post_to_classify)}")

# Vectorize the text content
X_post_text = vectorizer.transform(post_to_classify['originalContent'])

# Convert the additional features to a sparse matrix
X_post_additional = scipy.sparse.csr_matrix(post_to_classify[['contains_cursed_words', 'numberOfCursedWords']].values)

# Combine the text features and additional features
X_post = scipy.sparse.hstack([X_post_text, X_post_additional])

# Predict sentiment using the trained classifier
sentiment_predicted = clf.predict(X_post)

# Copy the original DataFrame to add the predicted sentiments
new_post_df = post_content.copy()

# Update the 'sentiment' column for the neutral posts with the predicted values
new_post_df.loc[new_post_df['sentiment'] == 'neutral', 'sentiment'] = sentiment_predicted

# Display the updated DataFrame
new_post_df.head()


Number of posts need to classify: 231684


,originalContent,sentiment,contains_cursed_words,numberOfCursedWords
0,vkl,negative,1,1
1,địt cụ thằng khải trần hô vni sập 900 100,negative,1,1
2,1 phát vn hai mấy,positive,0,0
3,má full tiềnđau,negative,0,0
4,gap khủng long,positive,0,0


Ta áp dụng mô hình phân loại cảm xúc đã huấn luyện để xác định cảm xúc cho các bài viết có nhãn "neutral". 

Trước tiên, tách các bài viết trung tính và vector hóa nội dung văn bản sử dụng TfidfVectorizer. Chuyển đổi thành ma trận thưa các đặc trưng bổ sung (số lượng từ không phù hợp và sự xuất hiện của từ không phù hợp). 

Các đặc trưng văn bản và bổ sung sau đó được kết hợp và dùng làm đầu vào cho mô hình để dự đoán cảm xúc. 

Cuối cùng, cột sentiment trong DataFrame gốc được cập nhật bằng các dự đoán này, giúp phân loại các bài viết trung tính thành tích cực hoặc tiêu cực.

In [104]:
# check the number of negative and positive posts before and after
print('Number of negative posts (before)', len(posts_df[posts_df['sentiment'] == 'negative']))
print('Number of positive posts(before)', len(posts_df[posts_df['sentiment'] == 'positive']))
print('Number of negative posts (after)', len(new_post_df[new_post_df['sentiment'] == 'negative']))
print('Number of positive posts(after)', len(new_post_df[new_post_df['sentiment'] == 'positive']))

Number of negative posts (before) 13317
Number of positive posts(before) 26076
Number of negative posts (after) 123447
Number of positive posts(after) 146910


So với phương pháp trước, số lượng bài viết thuộc hai lớp "tiêu cực" và "tích cực" hiện đã được cân bằng tốt hơn, với sự chênh lệch ở mức hợp lý(140910 với 123447). Kết quả này khẳng định rằng việc áp dụng kỹ thuật nhân đôi số lượng bài viết tiêu cực để cân bằng dữ liệu trong quá trình huấn luyện là một quyết định phù hợp, giúp mô hình học tốt hơn và giảm thiểu khả năng thiên vị.

In [105]:
#new_post_df.to_csv('new_posts_df.csv', index=False)
new_post_df.head()

,originalContent,sentiment,contains_cursed_words,numberOfCursedWords
0,vkl,negative,1,1
1,địt cụ thằng khải trần hô vni sập 900 100,negative,1,1
2,1 phát vn hai mấy,positive,0,0
3,má full tiềnđau,negative,0,0
4,gap khủng long,positive,0,0
